In [1]:
##
## This script is designed to extract a specific vertical profile from WRF
## A few lines are hard coded and will require further editing to be used with other cases
##

In [2]:
import pandas as pd
import os
from glob import glob
from os.path import join
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import netCDF4 as nc
import numpy as np
import xarray as xr
from scipy.interpolate import interp2d
from datetime import datetime, timedelta
from tqdm import tqdm
from pandas.plotting import register_matplotlib_converters
from wrf import getvar, ALL_TIMES,destagger, interplevel
register_matplotlib_converters()

In [3]:
params={     
    'axes.labelsize'  : '16',   
    'axes.titlesize'  : '16',  
    'xtick.labelsize' :'14',
    'ytick.labelsize' :'16',    
    'lines.linewidth' : '1' ,    
    'legend.fontsize' : '16', 
    'figure.figsize'   : '16, 9'    
}
plt.rcParams.update(params)

In [4]:
def nearest(array, number):
    nearest_index = np.where(np.abs(array - number) == np.nanmin(np.abs(array - number)))
    nearest_index = int(nearest_index[0])
    nearest_number = array[nearest_index]

    return (nearest_number, nearest_index)

def calc_wdir(u,v):
    wdir = np.arctan2(u,v)*180/np.pi
    wdir[wdir<0] = wdir[wdir<0]+360
    return wdir

In [5]:
def rh2qv(rh, press, air_temp):
    '''
    calculate QV using RH, pressure in hPa, and temperature in K
    '''
#     rh = np.float(sys.argv[1])
#     press = 1000
#     air_temp = 279.435791
    pt = air_temp*(1000/press)**(0.286)
    # Calculate specific humidity
    temp_degc = air_temp-273
    e_s = 0.61121*np.exp((18.678-temp_degc/234.5)*(temp_degc/(257.14+temp_degc))) # units: kPa

    w_s = 0.622*(e_s*10/press)

    w = rh/100*w_s
    qv = w/(1+w)
    return qv

def calc_uv(windspd, winddir):
    '''
    convert wind speed and wind direction to u & v
    '''
    rad = np.pi/180
    u, v = -windspd*np.sin(winddir * rad), -windspd*np.cos(winddir * rad)
    return(u, v)


In [6]:
# specify timestamp
ts = '200108051200'#33

In [7]:
wrf_path = '/data/WRF-model/V4.2/WRF-4.2/run/conus/'
wrf_file = 'wrfout_d04_2001-08-05_00:00:00'

ds_wrf = xr.open_dataset(wrf_path+wrf_file)
# add timestamp for selection
ds_wrf['Time'] = ds_wrf['XTIME']

# Read AWS

In [8]:
aws_path = '/data/WRF-model/WRF-validation/'
aws_file = 'CHA_2001.nc'
ds_aws = xr.open_dataset(aws_path+aws_file)
# CHA latitude/ longitude
lat_aws = -43.489 
lon_aws = 172.528

In [9]:
temp_aws = ds_aws['Tair'].sel(time=slice(ds_wrf['Time'][0], ds_wrf['Time'][-1]))+273.15
wspd_aws = ds_aws['wind_spd'].sel(time=slice(ds_wrf['Time'][0], ds_wrf['Time'][-1]))
wdir_aws = ds_aws['wind_dir'].sel(time=slice(ds_wrf['Time'][0], ds_wrf['Time'][-1]))
rh_aws = ds_aws['RH'].sel(time=slice(ds_wrf['Time'][0], ds_wrf['Time'][-1]))
u_aws, v_aws = calc_uv(wspd_aws, wdir_aws)

# Read CliFlo Upperair data

In [10]:
# CliFlo upper air obs
cliflo_path = '/data/CliFlo/CHA/'
wind_file = 'upper_air_2001-wind.genform1_proc'
temp_file = 'upper_air_2001-temperature.genform1_proc'

df_wind_upair = pd.read_csv(cliflo_path+wind_file, sep='\t', skiprows=8, header=0)
df_wind_upair.drop(df_wind_upair.tail(6).index,inplace=True)
df_wind_upair.replace('-',np.nan,inplace=True)

df_temp_upair = pd.read_csv(cliflo_path+temp_file, sep='\t', skiprows=8, header=0)
df_temp_upair.drop(df_temp_upair.tail(6).index,inplace=True)
df_temp_upair.replace('-',np.nan,inplace=True)

In [11]:
wind_dt = pd.to_datetime(df_wind_upair['Date(UTC)'],format='%Y,%m,%d,%H,%M')
temp_dt = pd.to_datetime(df_temp_upair['Date(UTC)'],format='%Y,%m,%d,%H,%M')

mask_wind = wind_dt == ts
mask_temp = temp_dt == ts

In [12]:
tt_upair = df_temp_upair['Tair(C)'][mask_temp].astype(np.float32)+273.15
pres_upair = df_temp_upair['Pressure(hPa)'][mask_temp].astype(np.float32)
thgt_upair = df_temp_upair['Height(m)'][mask_temp].astype(np.float32)

wt_upair = df_wind_upair['Tair(C)'][mask_wind].astype(np.float32)+273.15 
wdir_upair = df_wind_upair['Dir(degT)'][mask_wind].astype(np.float32)
whgt_upair = df_wind_upair['Height(m)'][mask_wind].astype(np.float32)
wspd_upair = df_wind_upair['Speed(m/s)'][mask_wind].astype(np.float32)
u_upair, v_upair = calc_uv(wspd_upair, wdir_upair)

# Read WRF

We will need:  
- vertical profile of u (U3d, U10)  
- vertical profile of v (V3d, V10) 
- vertical levels of u and v (PH, PHB) 
- surface pt (T2) 
- vertical gradient of pt (T2, theta)  
- vertical levels of pt (PH, PHB) 
- surface qv (QVAPOR)  
- vertical gradient of qv (QVAPOR)   
- vertical levels of qv (PH, PHB)  


In [13]:
# read latitudes and longitudes
lats_wrf = ds_wrf['XLAT'][0, :, 0]
lons_wrf = ds_wrf['XLONG'][0, 0, :]
_, idx_lat = nearest(lats_wrf, lat_aws)
_, idx_lon = nearest(lons_wrf, lon_aws)

In [14]:
## read variables
# potential temperature
theta_wrf = ds_wrf['T'][:, :, idx_lat, idx_lon] +300

# pressure
geop_wrf = ds_wrf['PH'] + ds_wrf['PHB']
press_wrf = (ds_wrf['P'][:, :, idx_lat, idx_lon] + ds_wrf['PB'][:, :, idx_lat, idx_lon]) *0.01

# temperature
tk_wrf = theta_wrf/((1000/press_wrf)**0.286)

# height
geoh_wrf = geop_wrf[:, :, idx_lat, idx_lon]/9.81

# wind
u_wrf = ds_wrf['U'][:, :, idx_lat, idx_lon]
v_wrf = ds_wrf['V'][:, :, idx_lat, idx_lon]
wspd_wrf = xr.ufuncs.sqrt(u_wrf**2+v_wrf**2)

# water vapour mixing ratio
qv_wrf = ds_wrf['QVAPOR'][:, :, idx_lat, idx_lon]

# soil moisture
smois_wrf = ds_wrf['SMOIS'][:, :, idx_lat, idx_lon]

# soil temperature
tslb_wrf = ds_wrf['TSLB'][:, :, idx_lat, idx_lon]

In [15]:
# layers for soil temperature and moisture calculation
# this shall be changed depending on different cases
dz_soil = np.array([0.01, 0.02, 0.04, 0.06, 0.14, 0.26, 0.54, 1.86])

# depth of soil layer
zs = ds_wrf['ZS'][0, :].data
# thickness of soil layer
dzs = ds_wrf['DZS'][0, :].data
# # landmask - 1 is land, 0 is water
# landmask = ds_wrf['LANDMASK'][0, idx_lat, idx_lon].data
# TMN - soil temperature at lower boundary
tmn = ds_wrf['TMN'][:, idx_lat, idx_lon]

init_soil_t = np.zeros(dz_soil.shape[0])
init_soil_m = np.zeros(dz_soil.shape[0])


    
init_soil_t[:] = np.interp(dz_soil, zs, tslb_wrf.sel(Time=ts).data)
init_soil_m[:] = np.interp(dz_soil, zs, smois_wrf.sel(Time=ts).data)
deep_soil_t = tmn.sel(Time=ts).data


In [16]:
# qvapor near surface from AWS
# use lowest level of upper air temperature observation
# WRF lowest level is above AWS height
qv_surface = rh2qv(rh_aws.sel(time=ts).data, pres_upair.iloc[0], tt_upair.iloc[0]) 
# use AWS temperature 
# assuming pressure at 2 m is similar to sea level
pt_surface = temp_aws.sel(time=ts).data 


In [17]:
# start to define vertical levels here
# we want to use upper air observations here

# 1. set uv profiles
# lowest level is 0 m with zero wind speed (remove if use flat terrain)
uv_heights = np.zeros(len(whgt_upair)+2)
uv_heights[1] = 10.0
uv_heights[2:] = np.array(whgt_upair)

u_profile = np.zeros_like(uv_heights)
u_profile[1] = u_aws.sel(time=ts).data
u_profile[2:] = np.array(u_upair)

v_profile = np.zeros_like(uv_heights)
v_profile[1] = v_aws.sel(time=ts).data
v_profile[2:] = np.array(v_upair)

In [18]:
# 2. potential temperature profile
pt_levels = np.zeros(len(thgt_upair)+1)
pt_levels[0] = 2.0
pt_levels[1:] = np.array(thgt_upair)

pt_profile = np.zeros(len(thgt_upair)+1)
pt_profile[0] = pt_surface
pt_profile[1:] = tt_upair*(1000/pres_upair)**(2/7)

In [20]:
zwrf = destagger(geoh_wrf.sel(Time=ts), stagger_dim=0)
qv = qv_wrf.sel(Time=ts)


In [22]:
hmax = thgt_upair.iloc[-1]
level_max = np.argwhere(zwrf<hmax)[-1,0]

In [23]:
# pt and qv profiles counting from 2 m
qv_levels = np.zeros(level_max+1)
qv_levels[0] = 2.0
qv_levels[1:] = zwrf[:level_max]

qv_profile = np.zeros(level_max+1)
qv_profile[0] = qv_surface
qv_profile[1:] = qv[:level_max]


In [24]:
def calc_gradient(z, vert, step):
        '''
        z : vertical heights
        vert : vertical profiles
        '''
        lvl = [z[0]]
        pr = [vert[0]]
        mask_lower = z<=100
        mask_upper = z>100
        for j in range(0,len(z[mask_lower])):
            if j==0:
                continue
            else:
                lvl.append(z[mask_lower][j])
                pr.append(vert[mask_lower][j])
        for i in range(0, len(z[mask_upper]), step):
#                 if i==0:
#                     continue
#                 else:
                    lvl.append(z[mask_upper][i])
                    pr.append(vert[mask_upper][i])
            
        # pr.append(vert[-1])
        gradient = []
        
        for j in range(1,len(pr)):
            # if j<len(pr):
            gradient.append( (pr[j]-pr[j-1])*100 / (lvl[j]-lvl[j-1]) )
            # else:
            #     gradient.append( (pr[j]-pr[j-1]) / (z[-1]-lvl[j]) )
        lvl.remove(lvl[-1])
        return(gradient, lvl)

In [25]:
pt_vertical_gradient, pt_gradient_level = calc_gradient(pt_levels, pt_profile, 2)
qv_vertical_gradient, qv_gradient_level = calc_gradient(qv_levels, qv_profile, 3)

In [26]:
def write_line(var_name, var_data):
    var_data = np.around(var_data, 8)
    if type(var_data) == np.float32 or type(var_data) == np.float64:
        line = var_name + ' = ' + '{:.6f}'.format(var_data) + ', '  +'\n' +'\n'
    else:
        line = var_name + ' = ' + ', '.join(map(str, var_data)) +'\n' +'\n'

    return line

In [27]:
with open('/data/WRF-model/WRF-validation/namelist_test', 'w') as namelist:
        namelist.write( write_line('uv_heights', uv_heights) )
        namelist.write( write_line('u_profile', u_profile) )
        namelist.write( write_line('v_profile', v_profile) )
        namelist.write( write_line('pt_surface', pt_surface) )
        namelist.write( write_line('pt_vertical_gradient', pt_vertical_gradient) )
        namelist.write( write_line('pt_vertical_gradient_level', pt_gradient_level) )
        namelist.write( write_line('q_surface', qv_surface) )
        namelist.write( write_line('q_vertical_gradient', qv_vertical_gradient) )
        namelist.write( write_line('q_vertical_gradient_level', qv_gradient_level) )
        namelist.write('& land_surface_model \n' )
        namelist.write( write_line('dz_soil', dz_soil) )
        namelist.write( write_line('soil_moisture', init_soil_m) )
        namelist.write( write_line('soil_temperature', init_soil_t) )
        namelist.write( write_line('deep_soil_temperature', deep_soil_t) )



In [28]:
nx   = 64          
ny   = 64  
nz   = 20    
dx   = 729.0  
dy   = 729.0  
dz   = 162.0
y = np.arange(dy/2,dy*(ny+0.5),dy)
x = np.arange(dx/2,dx*(nx+0.5),dx)
z = np.arange(dz/2, dz*nz, dz)
xu = x + np.gradient(x)/2
xu = xu[:-1]
yv = y + np.gradient(y)/2
yv = yv[:-1]
zw = z + np.gradient(z)/2
zw = zw[:-1]


In [29]:
interp_u = np.interp(z, uv_heights, u_profile)
interp_v = np.interp(z, uv_heights, v_profile)
interp_w = np.zeros_like(zw) # w is all zero
interp_pt = np.interp(z, pt_levels, pt_profile)
interp_qv = np.interp(z, qv_levels, qv_profile)

In [30]:
init_msoil_3d = np.tile(init_soil_m[:, np.newaxis, np.newaxis], (1, ny, nx))
init_tsoil_3d = np.tile(init_soil_t[:, np.newaxis, np.newaxis], (1, ny, nx))

In [31]:
nc_output = xr.Dataset()
nc_output['x'] = xr.DataArray(x, dims=['x'], attrs={'units':'m'})
nc_output['y'] = xr.DataArray(y, dims=['y'], attrs={'units':'m'})
nc_output['z'] = xr.DataArray(z, dims=['z'], attrs={'units':'m'})
nc_output['xu'] = xr.DataArray(xu, dims=['xu'], attrs={'units':'m'})
nc_output['yv'] = xr.DataArray(yv, dims=['yv'], attrs={'units':'m'})
nc_output['zw'] = xr.DataArray(zw, dims=['zw'], attrs={'units':'m'})
nc_output['zsoil'] = xr.DataArray(dz_soil, dims=['zsoil'], attrs={'units':'m'})
nc_output.to_netcdf("soil_test_dynamic")

In [32]:
nc_output['init_atmosphere_pt'] = xr.DataArray(interp_pt,dims=['z'],
         attrs={'units':'K', 'lod':np.int32(1)})
nc_output['init_atmosphere_qv'] = xr.DataArray(interp_qv,dims=['z'],
         attrs={'units':'kg/kg', 'lod':np.int32(1)})
nc_output['init_atmosphere_u'] = xr.DataArray(interp_u,dims=['z'],
         attrs={'units':'m/s', 'lod':np.int32(1)})
nc_output['init_atmosphere_v'] = xr.DataArray(interp_v,dims=['z'],
         attrs={'units':'m/s', 'lod':np.int32(1)})
nc_output['init_atmosphere_w'] = xr.DataArray(interp_w,dims=['zw'],
         attrs={'units':'m/s', 'lod':np.int32(1)})

In [33]:
nc_output['init_soil_m'] = xr.DataArray(init_msoil_3d, dims=['zsoil','y','x'], 
         attrs={'units':'m^3/m^3','lod':np.int32(2)}) 
nc_output['init_soil_t'] = xr.DataArray(init_tsoil_3d, dims=['zsoil','y','x'], 
         attrs={'units':'K', 'lod':np.int32(1)}) 

In [34]:
for var in nc_output.data_vars:
    encoding = {var: {'dtype': 'float32', '_FillValue': -9999, 'zlib':True}}
    nc_output[var].to_netcdf("soil_test_dynamic", mode ='a', encoding=encoding)

In [35]:
nc_output.close()